In [27]:
import numpy as np
import pandas as pd
from scipy.spatial.distance import cdist

import matplotlib.pyplot as plt
fig_width = (21-5)/2.54
import seaborn as sns
sns.set_context("paper")

import poissonlearning as pl

import storage
import plotting

plotting.setup(latex=True)

%load_ext autoreload
%autoreload 2

%matplotlib qt

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
SAVE_PLOTS = True
NUM_PLOTTING_POINTS = 1000

### Convenience functions

In [21]:
def compute_errors(experiments):
    for experiment in experiments:
        solution = experiment["solution"]

        reference_experiments = list(
            filter(
                lambda x: 
                    x["seed"] == experiment["seed"] 
                    and x["n"] == experiment["n"]
                    and x["bump"] == experiment["bump"]
                    and x["eps"] == experiment["eps"],
                experiments
            )
        )
        reference = max(reference_experiments, key=lambda x: x["p"])["solution"]

        error_L1 = np.abs(solution - reference).mean()
        experiment["error_L1"] = error_L1

In [22]:
experiments = storage.load_results(name="p_line", folder="../results")
compute_errors(experiments)

### Visualisation for various p

In [28]:
selected_experiments = list(
    filter(
        lambda x: 
            x["seed"] == 0 
            and x["n"] == 5000,
        experiments
    )
)
selected_experiments.sort(key=lambda x: x["p"])

In [34]:
sample_size = NUM_PLOTTING_POINTS

fig, ax_results = plt.subplots(1, 1, figsize=(fig_width, 0.45*fig_width))
ax_results.set_prop_cycle(plotting.line_cycler)

for i, ex in enumerate(selected_experiments, start=0):
    sample = ex["solution"].sample(sample_size, random_state=1).sort_index()
    ax_results.plot(sample, label=f"p={ex['p']}")

ax_results.legend(ncol=2)
ax_results.grid(linestyle="dashed")
ax_results.set_ylabel(r"$u_{n}^{(p)}$")
ax_results.set_ylim(-1.2, 1.2)

fig.tight_layout()

if SAVE_PLOTS:
    fig.savefig(f"../plots/p_line_convergence.pdf", bbox_inches="tight")

### Actual continuum solutions

In [25]:
def u_p(x, p, rho):
    return ((p-1)/p) * (rho ** (1/(p-1))) * np.sign(x) * (1 - ((1 - np.abs(x)) ** (p/(p-1))) )

In [35]:
sample_size = NUM_PLOTTING_POINTS

fig, ax_results = plt.subplots(1, 1, figsize=(fig_width, 0.45*fig_width))
ax_results.set_prop_cycle(plotting.line_cycler)

for i, ex in enumerate(selected_experiments, start=0):
    sample = ex["solution"].sample(sample_size, random_state=1).sort_index()
    y = u_p(sample.index, ex["p"], 0.5)
    ax_results.plot(sample.index, y, label=f"p={ex['p']}")

ax_results.legend(ncol=2)
ax_results.grid(linestyle="dashed")
ax_results.set_ylabel(r"$u^{(p)}$")
ax_results.set_ylim(-1.2, 1.2)

fig.tight_layout()

if SAVE_PLOTS:
    fig.savefig(f"../plots/p_line_convergence_continuum.pdf", bbox_inches="tight")